
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>

<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.5.2 - beta</p>
<p>Package compatibility: squid-py v0.5.11, keeper-contracts 0.9.0, utilities 0.2.1,
<p>Component compatibility: Brizo v0.3.2, Aquarius v0.2.1, Nile testnet smart contracts 0.8.6</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

# Getting Underway - Publishing assets
In this notebook, we will explore how to publish an Asset using Ocean Protocol. An Asset consists of several files
which are kept private, and optionally other links which are open (samples, descriptions, etc.).

A publisher will require access to two services;
1. A service to store the MetaData of the asset (part of the DDO) - 'Aquarius'
1. A service to manage permissioned access to the assets - 'Brizo'

The publishing of an asset consists of;
1. Preparing the asset files locally
1. Preparing the metadata of the asset
1. Uploading assets or otherwise making them available as URL's
1. Registering the metadata and service endpoints into Aquarius
1. Registering the asset into the Blockchain (into the DID Registry)

<p><img src="https://raw.githubusercontent.com/oceanprotocol/mantaray/develop/doc/img/jupyter_cell.png" alt="drawing" width="400" align="center"/></p>
<p><b>Overall client and service architecture</b></p>

### Section 0: Import modules, connect the Ocean Protocol API

In [ ]:
# Standard imports
import logging

# Import mantaray and the Ocean API (squid)
import random
import squid_py
from squid_py.ocean.ocean import Ocean
from squid_py.config import Config
import mantaray_utilities as manta_utils
from mantaray_utilities.user import password_map
from pprint import pprint
# Setup logging
manta_utils.logging.logger.setLevel('INFO')
from time import sleep
print("squid-py Ocean API version:", squid_py.__version__)

In [ ]:
# Get the configuration file path for this environment
CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))
logging.critical("Squid API version: {}".format(squid_py.__version__))

In [ ]:
# Instantiate Ocean with the default configuration file.
configuration = Config(CONFIG_INI_PATH)
ocn = Ocean(configuration)

### Section 1: A publisher account in Ocean

In [ ]:
# Get a publisher account

publisher_acct = manta_utils.user.get_account_by_index(ocn,0)

# path_passwords = manta_utils.config.get_project_path() / 'passwords.csv'
# passwords = manta_utils.user.load_passwords(path_passwords)
#
# publisher_acct = random.choice([acct for acct in ocn.accounts.list() if password_map(acct.address, passwords)])
# publisher_acct.password = password_map(publisher_acct.address, passwords)
# assert publisher_acct.password

In [ ]:
print("Publisher account address: {}".format(publisher_acct.address))
print("Publisher account Testnet 'ETH' balance: {:>6.1f}".format(ocn.accounts.balance(publisher_acct).eth/10**18))
print("Publisher account Testnet Ocean balance: {:>6.1f}".format(ocn.accounts.balance(publisher_acct).ocn/10**18))

Your account will need some Ocean Token to make real transactions, let's ensure that you are funded!

In [ ]:
# ensure Ocean token balance
# if ocn.accounts.balance(publisher_acct).ocn == 0:
#     ocn.accounts.request_tokens(publisher_acct, 100)

### Section 2: Create the Metadata for your asset
The metadata is a key-value set of attributes which describe your asset

A more complex use case is to manually generate your metadata conforming to Ocean standard, but for demonstration purposes,
a utility in squid-py is used to generate a sample Meta Data dictionary.

In [ ]:
# Get a simple example of Meta Data from the library directly
metadata = squid_py.ddo.metadata.Metadata.get_example()
print('Name of asset:', metadata['base']['name'])
# Print the entire (JSON) dictionary
pprint(metadata)

Note that the price is included in the Metadata! This will be purchase price you are placing on the asset. You can
Alter the metadata object at any time before publishing.

In [ ]:
print("Price of Asset:", metadata['base']['price'])
metadata['base']['price'] = 9
print("Updated price of Asset:", metadata['base']['price'])

Let's inspect another important component of your metadata - the actual asset files. The files of an asset are
described by valid URL's. You are responsible for ensuring the URL's are alive. Files may have additional
information, including a checksum, length, content type, etc.

In [ ]:
for i, file in enumerate(metadata['base']['files']):
    print("Asset link {}: {}".format( i, file['url']))

## Section 3 Publish the asset
With this metadata object prepared, we are ready to publish the asset into Ocean Protocol.

The result will be an ID string (DID) registered into the smart contract, and a DID Document stored in Aquarius.
The asset URLS's are encrypted upon publishing.

In [ ]:
ddo = ocn.assets.create(metadata, publisher_acct)
registered_did = ddo.did
print("New asset registered at", registered_did)

Inspect the new DDO. We can retrieve the DDO as a dictionary object, feel free to explore the DDO in the cell below!

In [ ]:
ddo_dict = ddo.as_dictionary()
print("DID:", ddo.did)
print("Services within this DDO:")
for svc in ddo_dict['service']:
    print(svc['type'], svc['serviceEndpoint'])

Note that the 'files' attribute has been modified - all URL's are now removed, and a new 'encryptedFiles'
attribute is created to store the actual URLs.

In [ ]:
for file_attrib in ddo.metadata['base']['files']:
    assert 'url' not in file_attrib
print("Encrypted files decrypt on purchase! Cipher text: [{}...] . ".format(ddo.metadata['base']['encryptedFiles'][:50]))

## Section 4: Verify your asset
Now, let's verify that this asset exists in the MetaData storage.

A call to assets.resolve() will call the Aquarius service and retrieve the DID Document

In [ ]:
#+attr_jupyter: some cell metadata stuff
#+attr_jupyter: some other metadata stuff

#TODO: Better handling based on reciept
print("Wait for the transaction to complete!")
sleep(10)

In [ ]:
ddo = ocn.assets.resolve(registered_did)
print("Asset '{}' resolved from Aquarius metadata storage: {}".format(ddo.did,ddo.metadata['base']['name']))

Similarly, we can verify that this asset is registered into the blockchain, and that you are the owner.

In [ ]:
# We need the pure ID string as in the DID registry (a DID without the prefixes)
asset_id = squid_py.did.did_to_id(registered_did)
owner = ocn._keeper.did_registry.contract_concise.getDIDOwner(asset_id)
print("Asset ID", asset_id, "owned by", owner)
assert str.lower(owner) == str.lower(publisher_acct.address)

Congratulations on publishing an Asset into Ocean Protocol!

Next, let's search for our assets in Ocean Protocol